### Сходство текстов
Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

* кошки (животные)

* UNIX-утилита cat для вывода содержимого файлов

* версии операционной системы OS X, названные в честь семейства кошачьих

Pадача — найти предложения, которые ближе всего по смыслу к другому выбранному предложению из текста. В качестве меры близости по смыслу будет использоваться косинусное расстояние.

In [1]:
import re
import numpy as np
import scipy.spatial.distance

In [2]:
with open('sentences.txt', 'r') as f:
    lines = f.readlines()
lines = [l.lower() for l in lines]
lines = [l.strip() for l in lines]

In [3]:
lines[:2]

['in comparison to dogs, cats have not undergone major changes during the domestication process.',
 'as cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.']

### Разобьем предложения на слова и подсчитаем количество вхождений каждого слова в наш текст.

In [4]:
tokens = []
for line in lines:
    for token in re.split('[^a-z]', line):
        if token != '':
            tokens.append(token)

In [5]:
tokens[:10]

['in',
 'comparison',
 'to',
 'dogs',
 'cats',
 'have',
 'not',
 'undergone',
 'major',
 'changes']

In [6]:
from collections import Counter

words_frequency = Counter(tokens)
words_frequency.most_common(5)

[('the', 20), ('of', 19), ('to', 14), ('and', 14), ('a', 13)]

Составим матрицу размера n * d, где n — число предложений, а d — число различных слов. Элемент с индексом (i, j) в этой матрице равен количеству вхождений j-го слова в i-е предложение. 


In [7]:
word_matrix = np.zeros((len(lines), len(words_frequency)))

for i in range(len(lines)):
    j = 0
    tokens_per_sentence = [t for t in re.split('[^a-z]', lines[i]) if t != '']
    for key in words_frequency.keys():
        word_matrix[i, j] = tokens_per_sentence.count(key)
        j += 1
word_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 1., 1.]])

Найдем похожие предложения с помощью косинусного расстояния

Воспользуемся следующей формулой: $$\begin{equation}
\cos ({\bf u},{\bf v})= {{\bf u} {\bf v} \over \|{\bf u}\| \|{\bf v}\|} = \frac{ \sum_{i=1}^{n}{{\bf u}_i{\bf v}_i} }{ \sqrt{\sum_{i=1}^{n}{({\bf u}_i)^2}} \sqrt{\sum_{i=1}^{n}{({\bf v}_i)^2}} }
\end{equation}$$


Из википедии (https://en.wikipedia.org/wiki/Cosine_similarity):

*In data analysis, cosine similarity is a measure of similarity between two sequences of numbers. For defining it, the sequences are viewed as vectors in an inner product space, and the cosine similarity is defined as the cosine of the angle between them, that is, the dot product of the vectors divided by the product of their lengths.*

*The resulting similarity ranges from −1 meaning exactly opposite, to 1 meaning exactly the same, with 0 indicating orthogonality or decorrelation, while in-between values indicate intermediate similarity or dissimilarity.*

*For example, in information retrieval and text mining, each word is assigned a different coordinate and a document is represented by the vector of the numbers of occurrences of each word in the document. Cosine similarity then gives a useful measure of how similar two documents are likely to be, in terms of their subject matter, and indepently of the length of the documents.*

In [8]:
example_sentence = 0 #будем искать предложения, похожие на lines[0]

distances = np.zeros(word_matrix.shape[0])

numerator = word_matrix[example_sentence] @ word_matrix.T
example_sentence_norm = np.sqrt((word_matrix[example_sentence] ** 2).sum())
denominator = np.sqrt((word_matrix ** 2).sum(axis=1)) * example_sentence_norm

distances = numerator / denominator

In [9]:
# a simpler unvectorized way of calculating cosine distances:

#for i in range(word_matrix.shape[0]):
#    distances[i] = scipy.spatial.distance.cosine(word_matrix[example_sentence, :], word_matrix[i, :])

In [10]:
distances[example_sentence]

1.0

Получим индексы *n* самых похожих на `example_sentence` предложений:

In [12]:
n = 2
closest_sentences = np.argsort(distances)[::-1][1:n+1]

In [13]:
print("Example sentence is:\n", lines[example_sentence])
print("Closest sentences are:\n")
for i in closest_sentences:
    print(lines[i])

Example sentence is:
 in comparison to dogs, cats have not undergone major changes during the domestication process.
Closest sentences are:

domestic cats are similar in size to the other members of the genus felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).
in one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.


Соберем это все в функцию для поиска похожих по содержанию предложений:

In [14]:
def find_similar(ex_sent, n=2):
    numerator = word_matrix[ex_sent] @ word_matrix.T
    ex_sent_norm = np.sqrt((word_matrix[ex_sent] ** 2).sum())
    denominator = np.sqrt((word_matrix ** 2).sum(axis=1)) * ex_sent_norm

    distances = numerator / denominator
    
    return np.argsort(distances)[::-1][1:n+1]

In [15]:
closest = find_similar(1, 3)
print("Example sentence is:\n", lines[1])
print("\nClosest sentences are:")
for i in closest:
    print(lines[i])

Example sentence is:
 as cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.

Closest sentences are:
in terms of legibility, a sequence of commands starting with cat and connected by pipes has a clear left-to-right flow of information.
when you type simply cat command without any arguments, it just receives the stdin content and displays it in the stdout.
as of mid 2010, some apple computers have firmware factory installed which will no longer allow installation of mac os x leopard.
